# Syn vs Nonsyn and Non-nonsense vs Nonsense mutation rates

## Based on positions, instead of codons!

This makes it easier to use these for FDR estimation.

In [7]:
%run "Header.ipynb"
%run "GeneUtils.ipynb"

In [8]:
import json
import pickle
import skbio
import pileup
from statistics import median
from math import log
from collections import defaultdict
from matplotlib import pyplot
from parse_sco import parse_sco

## Percentages (values of $p$) for naive mutation calling

### Values we use for plotting
Currently the same as in the codon mutation notebook. I guess these should be a shared variable, ideally? But that might be too much work for its own good.

In [9]:
percentages = [2, 1, 0.5, 0.25, 0.15]

### Full values, used for plotting a FDR curve elsewhere

Copied from the target/decoy notebook. may be nice to generalize to shared code too i guess

In [10]:
# Percentages go from 4.99%, 4.98%, ..., 0.16%, 0.15%
full_percentages = [p / 100 for p in range(15, 500, 1)][::-1]
print(f"First two percentages: {full_percentages[:2]}")
print(f"Last two percentages: {full_percentages[-2:]}")
print(f"Number of percentages: {len(full_percentages):,}")

First two percentages: [4.99, 4.98]
Last two percentages: [0.16, 0.15]
Number of percentages: 485


In [11]:
# Sanity check -- we can just compute this data for all of the stuff in full_percentages
# and then for the stuff we wanna plot in this notebook just focus on the values in percentages
for p in percentages:
    assert p in full_percentages, f"{p}% not in full_percentages"

## Compute possible (non)synonymous and (non)nonsense mutations, in general

Not specific to a given MAG -- just in the context of all possible codons, using the standard genetic code.

In [12]:
codons = []
dna = "ACGT"
for x in dna:
    for y in dna:
        for z in dna:
            codons.append(x+y+z)
            
# Maps the 64 codons --> [1, 2, 3] --> an integer 0, 1, 2, or 3, indicating how many of the 3 mutations
# from this CP in this codon into another nucleotide are synonymous (si) or nonsynonymous (ni)
codon2cp2si = defaultdict(dict)
codon2cp2ni = defaultdict(dict)

# Like above, but for non-nonsense (nnsi) or nonsense (nsi) mutations and only for the 61 sense codons
codon2cp2nnsi = defaultdict(dict)
codon2cp2nsi = defaultdict(dict)

stop_codons = []

for c in codons:
    is_not_stop = not (str(skbio.DNA(c).translate()) == "*")
    if not is_not_stop:
        stop_codons.append(c)
    for pos in [0, 1, 2]:
        si = 0
        ni = 0
        nnsi = 0
        nsi = 0
        for alt_nt in set(dna) - set(c[pos]):
            alt_codon = c[:pos] + alt_nt + c[pos + 1:]
            aa1 = str(skbio.DNA(c).translate())
            aa2 = str(skbio.DNA(alt_codon).translate())
            if aa1 == aa2:
                si += 1
                if is_not_stop:
                    nnsi += 1
            else:
                ni += 1
                if is_not_stop:
                    if aa2 == "*":
                        nsi += 1
                    else:
                        nnsi += 1
                    
        assert si + ni == 3
        codon2cp2si[c][pos + 1] = si
        codon2cp2ni[c][pos + 1] = ni
        
        if is_not_stop:
            assert nsi + nnsi == 3
            codon2cp2nnsi[c][pos + 1] = nnsi
            codon2cp2nsi[c][pos + 1] = nsi
            
assert len(codon2cp2si) == len(codon2cp2ni) == 64
assert len(codon2cp2nnsi) == len(codon2cp2nsi) == 61
assert len(stop_codons) == 3

## Use these numbers to compute $\sum_i S_i$ (and $N_i$, $NNS_i$, $NS_i$) for all MAGs

In [ ]:
# seq --> list of 1-indexed positions where a mutation here can be nonsyn or nonsense
seq2poss_nonsyn_positions = {}
seq2poss_nonsense_positions = {}

# Total poss numbers of single-nucleotide (non)synonymous and (non)nonsense mutations throughout the genomes
seq2poss_si = defaultdict(int)
seq2poss_ni = defaultdict(int)
seq2poss_nnsi = defaultdict(int)
seq2poss_nsi = defaultdict(int)

for seq in SEQS:
    df = parse_sco(f"../seqs/genes/{seq}.sco")
    fasta = skbio.DNA.read(f"../seqs/{seq}.fasta")
    # Figure out quickly which positions are in multiple genes, so we can ignore them
    pos_to_genes = get_parent_gene_info_of_many_positions(df)
    
    poss_nonsyn_pos = []
    poss_nonsense_pos = []
    
    for gene in df.itertuples():
        print(
            f"{seq2name[seq]}: On gene {gene.Index:,} / {len(df.index):,}. "
            f"So far {len(poss_nonsyn_pos):,} poss nonsyn and {len(poss_nonsense_pos):,} "
            f"poss nonsense positions."
        )
        # The order of positions we go through is dependent on gene orientation:
        # left --> right if this is a "+" strand gene,
        # right --> left if this is a "-" strand gene.
        pos_interval = get_pos_interval_from_gene(gene)
        
        # We'll update these values as we walk through the gene.
        # curr_codon_cp1_pos indicates CP1 (the left end for + genes, the right end for - genes)
        # of the current gene. It's 1-indexed, along with the gene coordinates.
        curr_codon_cp1_pos = pos_interval[0]
        # Always either 1, 2, or 3
        cp = 1
        
        for pos in pos_interval:
            if cp == 1:
                # It's easier to sync this at the top rather than bottom of the loop because the positions
                # in pos_interval could be increasing or decreasing depending on the gene's strand
                curr_codon_cp1_pos = pos
            
            num_parent_genes = len(pos_to_genes[pos])
            if num_parent_genes == 1:
                
                if gene.Strand == "+":
                    # The skbio sequence is 0-indexed, so we gotta subtract by 1
                    parent_codon_fasta = fasta[curr_codon_cp1_pos - 1: curr_codon_cp1_pos + 2]
                elif gene.Strand == "-":
                    parent_codon_fasta = fasta[curr_codon_cp1_pos - 3: curr_codon_cp1_pos].reverse_complement()
                else:
                    raise ValueError(f"Invalid strand: {gene.Strand}")
                
                parent_codon_str = str(parent_codon_fasta)
                seq2poss_si[seq] += codon2cp2si[parent_codon_str][cp]
                seq2poss_ni[seq] += codon2cp2ni[parent_codon_str][cp]
                if codon2cp2ni[parent_codon_str][cp] > 0:
                    poss_nonsyn_pos.append(pos)
                    
                if parent_codon_str not in stop_codons:
                    seq2poss_nnsi[seq] += codon2cp2nnsi[parent_codon_str][cp]
                    seq2poss_nsi[seq] += codon2cp2nsi[parent_codon_str][cp]
                    if codon2cp2nsi[parent_codon_str][cp] > 0:
                        poss_nonsense_pos.append(pos)
                    
            cp = next_cp(cp)
    seq2poss_nonsyn_positions[seq] = poss_nonsyn_pos
    seq2poss_nonsense_positions[seq] = poss_nonsense_pos

In [14]:
for seq in SEQS:
    n = len(seq2poss_nonsyn_positions[seq])
    ns = len(seq2poss_nonsense_positions[seq])
    ratio = n / ns
    print(
        f"{seq}: {n:,} possible nonsyn and "
        f"{ns:,} possible nonsense mutations. Ratio of #nonsyn to #nonsense: {ratio:.2f}"
    )

edge_6104: 1,045,732 possible nonsyn and 132,806 possible nonsense mutations. Ratio of #nonsyn to #nonsense: 7.87
edge_1671: 1,685,006 possible nonsyn and 240,546 possible nonsense mutations. Ratio of #nonsyn to #nonsense: 7.00
edge_2358: 1,917,615 possible nonsyn and 227,810 possible nonsense mutations. Ratio of #nonsyn to #nonsense: 8.42
